## ECON 446 Final Project
Due Date: June 12th, 2024 <br>
Group Members: Marcy Guo, Vina Sun

1.) Find some data table that updates periodically.

2.) Pull in the data using python and store to a google sheet.

3.) Make it text message you if there is a change in the data.

4.) Set it up as a cloud function. 

5.) Post to the cloud and schedule daily. 

6.) Record a video showing this pipeline in under 3 minutes.
- Run your GCloud function and show that it updates the google sheet

- Have everyone from your group talk through a different section

- Make 1 slide showing the useful business / academic implications of your data collection.

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from twilio.rest import Client

##### 1. Data table: USD/CNY Exchange Rate Historical Data (https://finance.yahoo.com/quote/CNY%3DX/history/)

In [2]:
# Set up Chrome driver
service = Service('/usr/local/bin/chromedriver')
options = Options()
#options.add_argument('--headless')  # Uncomment if you want to run in headless mode
driver = webdriver.Chrome(service=service, options=options)

try:
    # Navigate to the URL
    url = 'https://finance.yahoo.com/quote/CNY%3DX/history/'
    driver.get(url)

    # Increase wait time
    wait = WebDriverWait(driver, 20)

    # Ensure page is fully loaded by waiting for the "Related Tickers" section
    wait.until(EC.presence_of_element_located((By.XPATH, '//section[@data-testid="related-tickers"]')))

    # Wait for the historical data table to load
    table = driver.find_element(By.XPATH, '//table[contains(@class, "svelte-ewueuo")]')

    # Extract the data from the table
    rows = table.find_elements(By.XPATH, './/tbody/tr')
    for row in rows:
        cells = row.find_elements(By.XPATH, './/td')
        if len(cells) < 5:  # Ensure row has enough cells
            continue
        date = cells[0].text
        open_price = cells[1].text
        high_price = cells[2].text
        low_price = cells[3].text
        close_price = cells[4].text
        print(f'Date: {date}, Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}')

finally:
    # Close the driver
    driver.quit()


Date: Jun 6, 2024, Open: 7.2442, High: 7.2442, Low: 7.2442, Close: 7.2442
Date: Jun 6, 2024, Open: 7.2467, High: 7.2467, Low: 7.2416, Close: 7.2467
Date: Jun 5, 2024, Open: 7.2420, High: 7.2468, Low: 7.2420, Close: 7.2420
Date: Jun 4, 2024, Open: 7.2420, High: 7.2420, Low: 7.2420, Close: 7.2420
Date: Jun 3, 2024, Open: 7.2410, High: 7.2420, Low: 7.2410, Close: 7.2410
Date: May 31, 2024, Open: 7.2335, High: 7.2335, Low: 7.2335, Close: 7.2335
Date: May 30, 2024, Open: 7.2494, High: 7.2494, Low: 7.2335, Close: 7.2494
Date: May 29, 2024, Open: 7.2450, High: 7.2450, Low: 7.2450, Close: 7.2450
Date: May 28, 2024, Open: 7.2415, High: 7.2415, Low: 7.2415, Close: 7.2415
Date: May 27, 2024, Open: 7.2415, High: 7.2415, Low: 7.2415, Close: 7.2415
Date: May 24, 2024, Open: 7.2415, High: 7.2415, Low: 7.2415, Close: 7.2415
Date: May 23, 2024, Open: 7.2403, High: 7.2444, Low: 7.2403, Close: 7.2403
Date: May 22, 2024, Open: 7.2376, High: 7.2403, Low: 7.1631, Close: 7.2376
Date: May 21, 2024, Open: 7.23

##### 2. Pull in the data using python and store to a google sheet.

In [4]:
# Set up Chrome driver
service = Service('/usr/local/bin/chromedriver')  # Adjust path to your chromedriver
options = Options()
# options.add_argument('--headless')  # Uncomment if you want to run in headless mode
driver = webdriver.Chrome(service=service, options=options)

try:
    # Navigate to the URL
    url = 'https://finance.yahoo.com/quote/CNY%3DX/history/'
    driver.get(url)

    # Increase wait time
    wait = WebDriverWait(driver, 10)

    # Ensure page is fully loaded by waiting for the "Related Tickers" section
    wait.until(EC.presence_of_element_located((By.XPATH, '//section[@data-testid="related-tickers"]')))

    # Wait for the historical data table to load
    table = driver.find_element(By.XPATH, '//table[contains(@class, "svelte-ewueuo")]')

    # Extract the data from the table
    rows = table.find_elements(By.XPATH, './/tbody/tr')
    data = []
    for row in rows:
        cells = row.find_elements(By.XPATH, './/td')
        if len(cells) < 5:  # Ensure row has enough cells
            continue
        date = cells[0].text
        open_price = cells[1].text
        high_price = cells[2].text
        low_price = cells[3].text
        close_price = cells[4].text
        data.append([date, open_price, high_price, low_price, close_price])
        #print(f'Date: {date}, Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}')
finally:
    # Close the driver
    print("Data extracted successfully")
    driver.quit()

# Convert data to DataFrame
df = pd.DataFrame(data, columns=["Date", "Open", "High", "Low", "Close"])

# Google Sheets API setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/marcyguo/Desktop/ECON 446/Final/cloud-project.json', scope)  # Adjust path to your credentials file
client = gspread.authorize(creds)

# Open a Google Sheet
sheet = client.open("ECON 446 Final Project").sheet1  # Adjust to your Google Sheet name

# Clear existing data
sheet.clear()

# Update with new data
sheet.update([df.columns.values.tolist()] + df.values.tolist())

print("Data updated in Google Sheet")


Data extracted successfully
Data updated in Google Sheet


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import gspread

# Set up Chrome driver
service = Service('/usr/local/bin/chromedriver')  # Adjust path to your chromedriver
options = Options()
# options.add_argument('--headless')  # Uncomment if you want to run in headless mode
driver = webdriver.Chrome(service=service, options=options)

try:
    # Navigate to the URL
    url = 'https://finance.yahoo.com/quote/CNY%3DX/history/'
    driver.get(url)

    # Increase wait time
    wait = WebDriverWait(driver, 10)

    # Ensure page is fully loaded by waiting for the "Related Tickers" section
    wait.until(EC.presence_of_element_located((By.XPATH, '//section[@data-testid="related-tickers"]')))

    # Wait for the historical data table to load
    table = wait.until(EC.presence_of_element_located((By.XPATH, '//table[contains(@class, "svelte-ewueuo")]')))

    # Extract the data from the table
    rows = table.find_elements(By.XPATH, './/tbody/tr')
    data = []
    for row in rows:
        cells = row.find_elements(By.XPATH, './/td')
        if len(cells) < 6:  # Ensure row has enough cells
            continue
        date = cells[0].text
        open_price = cells[1].text
        high_price = cells[2].text
        low_price = cells[3].text
        close_price = cells[4].text
        adj_close_price = cells[5].text
        volume = cells[6].text if len(cells) > 6 else '-'
        data.append([date, open_price, high_price, low_price, close_price, adj_close_price, volume])
finally:
    # Close the driver
    print("Data extracted successfully")
    driver.quit()

# Convert data to DataFrame
df = pd.DataFrame(data, columns=["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])

# Google Sheets API setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/marcyguo/Desktop/ECON 446/Final/cloud-project.json', scope)  # Adjust path to your credentials file
client = gspread.authorize(creds)

# Open a Google Sheet
sheet = client.open("ECON 446 Final Project").sheet1  # Adjust to your Google Sheet name

# Clear existing data
sheet.clear()

# Update with new data
sheet.update([df.columns.values.tolist()] + df.values.tolist())

print("Data updated in Google Sheet")

Data extracted successfully
Data updated in Google Sheet


##### 3. Make it text message you if there is a change in the data.

In [2]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from twilio.rest import Client
import chromedriver_autoinstaller
from datetime import datetime

# Function to set up Chrome driver
def set_up_driver():
    chromedriver_autoinstaller.install()  # Automatically install the correct ChromeDriver version
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

# Function to scrape data from Yahoo Finance
def scrape_data(driver):
    url = 'https://finance.yahoo.com/quote/CNY%3DX/history/'
    driver.get(url)
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.XPATH, '//section[@data-testid="related-tickers"]')))
    table = driver.find_element(By.XPATH, '//table[contains(@class, "svelte-ewueuo")]')
    rows = table.find_elements(By.XPATH, './/tbody/tr')
    data = []
    for row in rows:
        cells = row.find_elements(By.XPATH, './/td')
        if len(cells) < 5:
            continue
        date = cells[0].text
        open_price = cells[1].text
        high_price = cells[2].text
        low_price = cells[3].text
        close_price = cells[4].text
        data.append([date, open_price, high_price, low_price, close_price])
    return pd.DataFrame(data, columns=["Date", "Open", "High", "Low", "Close"])

# Google Sheets API setup
def set_up_gspread():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/marcyguo/Desktop/ECON 446/Final/gemini-pr-8b6c22e2d8b5.json', scope)
    client = gspread.authorize(creds)
    return client

# Function to update Google Sheet
def update_google_sheet(data):
    client = set_up_gspread()
    sheet = client.open("ECON 446 Final Project").sheet1  # Adjust to your Google Sheet name

    # Convert scraped data to DataFrame
    df = pd.DataFrame(data, columns=["Date", "Open", "High", "Low", "Close"])

    # Read existing data
    existing_data = sheet.get_all_values()

    # Convert existing data to DataFrame
    if len(existing_data) > 1:
        existing_df = pd.DataFrame(existing_data[1:], columns=existing_data[0])
    else:
        existing_df = pd.DataFrame(columns=df.columns)

    # Update existing rows and find new rows
    updated_df = existing_df.copy()
    new_rows = []

    for _, row in df.iterrows():
        date = row["Date"]
        if date in existing_df["Date"].values:
            updated_df.loc[updated_df["Date"] == date] = row
        else:
            new_rows.append(row)

    # Add new rows to the top of the updated data
    if new_rows:
        new_rows_df = pd.DataFrame(new_rows, columns=df.columns)
        updated_df = pd.concat([new_rows_df, updated_df], ignore_index=True)

    # Update Google Sheet with new data
    sheet.clear()
    sheet.append_row(updated_df.columns.values.tolist())
    sheet.append_rows(updated_df.values.tolist())
    print("Data updated in Google Sheet")

    # Send SMS notification
    #send_sms("The Google Sheet has been updated with new data.")

# Function to send SMS notification
def send_sms(body):
    ACCOUNT_SID = 'ACaf997ebd288f0ee3fcd9e78de114bed2'
    AUTH_TOKEN = 'c3599c800f9f0e07c9d2a5d7bf22a6c0'
    TWILIO_PHONE_NUMBER = '+18558274671'
    TO_PHONE_NUMBER = '+19842615503'

    client = Client(ACCOUNT_SID, AUTH_TOKEN)

    try:
        message = client.messages.create(
            body=body,
            from_=TWILIO_PHONE_NUMBER,
            to=TO_PHONE_NUMBER
        )
        print("SMS sent successfully")
    except Exception as e:
        print(f"Failed to send SMS: {e}")

# Main function to scrape data, update Google Sheet, and send SMS notification
def main():
    driver = set_up_driver()
    try:
        data = scrape_data(driver)
        update_google_sheet(data)
    finally:
        driver.quit()

# Run the main function
if __name__ == "__main__":
    main()

InvalidJSONError: Out of range float values are not JSON compliant

##### 4. Set it up as a cloud function. 

In [14]:
# Local Function Test
from bs4 import BeautifulSoup
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

def web_scrape_to_sheets(request):
    # Scrape the website with headers
    url = 'https://finance.yahoo.com/quote/CNY%3DX/history/'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Print the HTML
    #print(soup.prettify())
    
    # Parse the data
    data = []
    table = soup.find('table', {'class': 'svelte-ewueuo'})
    if table is None:
        print('No table found')
        return data

    tbody = table.find('tbody')
    if tbody is None:
        print('No tbody found in table')
        return data

    for row in tbody.find_all('tr', {'class': 'svelte-ewueuo'}):
        cols = row.find_all('td', {'class': 'svelte-ewueuo'})
        if len(cols) == 7:  # Ensure that the row has the correct number of columns
            date = cols[0].text.strip()
            open_price = cols[1].text.strip()
            high = cols[2].text.strip()
            low = cols[3].text.strip()
            close = cols[4].text.strip()
            adj_close = cols[5].text.strip()
            volume = cols[6].text.strip()
            data.append([date, open_price, high, low, close, adj_close, volume])
        else:
            print(f'Row with {len(cols)} columns found, expected 7')
    return data

data = web_scrape_to_sheets(None)

# Convert data to DataFrame
df = pd.DataFrame(data, columns=["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])

# Google Sheets API setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('cloud-project.json', scope)  # Adjust path to your credentials file
client = gspread.authorize(creds)

# Open a Google Sheet
sheet = client.open("ECON 446 Final Project").sheet1  # Adjust to your Google Sheet name

# Clear existing data
sheet.clear()

# Update with new data
sheet.update([df.columns.values.tolist()] + df.values.tolist())

print("Data updated in Google Sheet")


Data updated in Google Sheet


In [43]:
import requests

# URL of your deployed Cloud Function
url = 'https://us-central1-could-project-423421.cloudfunctions.net/web_scrape_to_sheets'

# Send a POST request
response = requests.post(url)

# Print the response
print(response.status_code)
print(response.text)  # Use response.text instead of response.json()


200
{"message":"Data updated successfully"}

